In [ ]:
%pip install --upgrade DBcm

In [ ]:
"""following code is what i use to generate data, this  will code will be used to set up data for graphs and data base in the data visualisation"""

In [178]:
import DBcm
import random

In [3]:
creds = {
    'host': 'localhost',
    'database': 'BlackJackDB',
    'user': 'josh',
    'password': 'password'
}

In [4]:
def random_gamer():    
    with DBcm.UseDatabase(creds) as db:
        SQL = """
        select gamertag from players
        """
        db.execute(SQL)
        tags = db.fetchall()
        tags = [row[0] for row in tags] 
        who = random.choice(tags)
    return who

In [21]:
def update_games_database(gamertag, outcome, isTwentyOne):
    """ updates the game data"""
    SQL = """
        insert into games
        (gamertag, outcome,21s)
        values
        (%s,%s,%s)      
        """


    with DBcm.UseDatabase(creds) as db:
        db.execute(SQL, (gamertag, outcome, isTwentyOne))

        SQL = """select * from games"""
        db.execute(SQL)
        results = db.fetchall()

In [155]:

def generate_game_data():
    """ populates database"""
    for _ in range(50):
        gamer = random_gamer()
        outcome = random.choice(["Win", "Loss"])

        if(outcome == "Win"):
        
            twentyOne = random.choice(["yes", "no", "no"]) # just to make it less likely they wil get 21
        else:
            twentyOne = "no"
        
      
        update_games_database(gamer, outcome, twentyOne)
    ## print(results[-3:])

In [71]:
def get_winrate(player):
    "gets a players win rate"
    with DBcm.UseDatabase(creds) as db: 
        SQL = """
        select count(*) from games 
        where gamertag = %s and outcome = "Win"
        """
        db.execute(SQL,(player,))
        results = db.fetchone()
        wins = results[0]

        SQL = """
        select count(*) from games 
        where gamertag = %s and outcome = "Loss"
        """
        db.execute(SQL,(player,))
        results = db.fetchone()
        losses = results[0]

        winRate = (wins/(wins + losses)) * 100

        return winRate

In [175]:
def get_highest_win_streak(player):
    """gets a players highest win streak"""
    with DBcm.UseDatabase(creds) as db: 

        SQL = """select HighestWinStreak from gamestats where gamertag = %s"""
        db.execute(SQL,(player, ))
        currentHighScore = db.fetchone()
        

        SQL = """select outcome from games where gamertag = %s """
        db.execute(SQL,(player,))
        results = db.fetchall()
        
        highestWinStreak = 0; 
        winStreak =0; 
        
        for result in results:
            if "Loss" in result:
                if(highestWinStreak < winStreak):
                     highestWinStreak = winStreak
                winStreak = 0
            else:
                winStreak +=1
            
        if( winStreak != 0):
            if( winStreak > highestWinStreak):
                highestWinStreak = winStreak


    return highestWinStreak

In [137]:
def get_21s(player):
    """gets a player number of 21s from table"""
    with DBcm.UseDatabase(creds) as db: 
        SQL = """select 21s from games where gamertag = %s """
        db.execute(SQL,(player,))
        results = db.fetchall()
        twentyOnesResults = 0
        for result in results:
            if "yes" in result:
                twentyOnesResults+=1
    return twentyOnesResults


In [197]:
def update_gameStats(winrate, high_streak, twenty_ones, gamertag):
    """updates the stats table if there is a gamer tag exist and inserts if none """
    with DBcm.UseDatabase(creds) as db: 
            ## check if user exist first 
            SQL = """select gamertag from gamestats where gamertag = %s """
            db.execute(SQL,(gamertag,))
            result = db.fetchone()
            print(result)
            if result is None:
                SQL = """ 
                    insert into gamestats
                    (gamertag, winrate, HighestWinStreak, 21s)
                    values
                    (%s,%s, %s, %s)      
                    """
                db.execute(SQL,(gamertag,winrate,high_streak,twenty_ones))
            else:
                SQL = """
                update gamestats 
                set winrate = %s, HighestWinStreak = %s, 21s =%s 
                where gamertag =%s
                """
                db.execute(SQL,(winrate,high_streak,twenty_ones, gamertag))
                  

        
            

In [192]:
with DBcm.UseDatabase(creds) as db:
    SQL = """select gamertag from gamestats where gamertag = "JDawg" """
    db.execute(SQL)
    result = db.fetchone()
    print(result)
    if result == None:
        print("work")

None
work


In [152]:
def get_player_names():
    """gets a list of players names"""
    with DBcm.UseDatabase(creds) as db:
        SQL = """
        select gamertag from players
        """
        db.execute(SQL)
        tags = db.fetchall()
        tags = [row[0] for row in tags] 
    return tags

In [180]:
generate_game_data()
players = get_player_names()
for player in players:

    update_gameStats(get_winrate(player), get_highest_win_streak(player), get_21s(player), player)